### 01_comparing_KBI_correction_methods

comparing various kbi correction methods and their resulting KBI values in the thermodynamic limit

In [2]:
run_example = False  # Set to True to run the example in Jupyter


import numpy as np
import pandas as pd

from picmol import KBI, KBIPlotter
from picmol import load_molecular_properties, add_molecule

if run_example:
  prj_dir = '/path/to/mixture/directory/'
  pure_component_dir = '/path/to/pure/component/directory/'

  # first check that molecules are in picmol dataset
  mol_props = load_molecular_properties('mol_id')
  mol_props.index # check that molecular ids (name in .top file) are present

  # if molecule(s) are not there, then add them with add_molecule
  add_molecule(mol_name='fig/mol/name', mol_id='mol_id/from/.top', mol_class='solute/solvent/extractant/modifier/', smiles='smiles/str', density='g/mL/at/RT/if/known') # if density is not entered, it will be calculated from RDKit's molar volume estimation, and if provided it will be used to calculate experimental molar volume

  # check that molecules are now provided in dataset
  mol_props = load_molecular_properties('mol_id')
  mol_props.index 


  # initialize kbi class
  kbi_obj = KBI(
    prj_path = prj_dir, # location for project to analyze
    pure_component_path = pure_component_dir, # location of pure component directory
    rdf_dir = 'rdf_files', # name for rdf file directory, must be same for all systems
    kbi_method = 'adj', # which kbi correction method to use
    rkbi_min = 0.75, # fraction of rdf to start at for thermo limit extrapolation
    avg_start_time = 100, # when to start averaging properties in .edr file
    solute_mol = '/enter/solute/mol_id/here/', # should be same name as in .top file
    geom_mean_pairs = [], # if desired, enter list of list of molecule pair to use as geom means
  )

  # run the kbi analysis
  kbi_obj.run()

  # first check extrapolation to the thermodynamic limit to make sure the appropriate range is found
  kbi_plotter = KBIPlotter(kbi_obj)
  kbi_plotter.make_figures()
  # if fit is not linear, change the 'rkbi_min' value & rerun analysis


  # view activity coefficients
  kbi_obj.gammas    

  # view molecule order corresponding to activity coefficient columns,
  kbi_obj.unique_mols


  # compare the various kbi correction methods
  kbi_dict = {}
  kbi_methods = ['raw', 'adj', 'gv', 'kgv']
  for m, method in enumerate(kbi_methods):
    # create kbi object for each method
    kbi_obj = KBI(
      prj_path = prj_dir, # location for project to analyze
      pure_component_path = pure_component_dir, # location of pure component directory
      rdf_dir = 'rdf_files', # name for rdf file directory, must be same for all systems
      kbi_method = method, # which kbi correction method to use
      rkbi_min = 0.75, # fraction of rdf to start at for thermo limit extrapolation
      avg_start_time = 100, # when to start averaging properties in .edr file
      solute_mol = '/enter/solute/mol_id/here/', # should be same name as in .top file
      geom_mean_pairs = [], # if desired, enter list of list of molecule pair to use as geom means
    )
    # run kbi analysis
    kbi_obj.run()

    # add kbi_obj to dictionary
    kbi_dict[method] = kbi_obj


  import matplotlib.pyplot as plt
  %matplotlib inline

  def add_kbi_method_to_plot(method, axs):
    kbi_obj = kbi_dict[method]
    ij = 0
    for i, mol_1 in enumerate(kbi_obj._top_unique_mols):
      for j, mol_2 in enumerate(kbi_obj._top_unique_mols):
        if i <= j:
          axs.scatter(kbi_obj.z[:,kbi_obj.solute_loc], kbi_obj.df_kbi[f'G_{mol_1}_{mol_2}_cm3_mol'], c=colors[ij], marker='s', linewidth=1.8, label=f'{kbi_obj.mol_name_dict[mol_1]}-{kbi_obj.mol_name_dict[mol_2]}')
          ij += 1

  # create figure for comparing the kbi methods
  fig, ax = plt.subplots(nrow=1, ncol=len(kbi_methods), figsize=(12, 3.5), sharex=True, sharey=True)
  colors = plt.cm.jet(np.linspace(0,1,len(self.kbi_model._top_unique_mols)+1))

  for m, method in enumerate(kbi_methods):
    ax[m].set_title(f'KBI Method: {method}')
    add_kbi_method_to_plot(method, ax[m])
    ax[m].set_xlabel(f'${x_lab}_{{{kbi_dict["adj"].solute_name}}}$')

  ax[0].legend(fontsize=11, labelspacing=0.5, frameon=True, edgecolor='k', framealpha=0.5)
  ax[0].set_xlim(-0.05, 1.05)
  ax[0].set_xticks(ticks=np.arange(0,1.1,0.1))
  ax[0].set_ylabel(f'$G_{{ij}}^{{\infty}}$ [cm$^3$ mol$^{{-1}}$]')